In [1]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

In [2]:
dev = pd.read_csv('../data/validation.csv')
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
avgCTR = train[train['click'] == 1].shape[0] / train.shape[0]

In [4]:
click = dev['click'].values.ravel()
payprice = dev['payprice'].values.ravel()
slotprice = dev['slotprice'].values.ravel()

In [5]:
pred_gl = pd.read_csv('../prediction/pred_dev_gl.csv').values.ravel()
pred_esm = pd.read_csv('../prediction/pred_dev_esm.csv').values.ravel()
pred_test = pd.read_csv('../prediction/pred_test.csv', index_col='bidid')

In [21]:
y_pred_valid = pd.read_csv('../../y_pred_valid.csv', index_col=0).values
y_pred_test = pd.read_csv('../../y_pred_test.csv', index_col=0).values

In [11]:
y_pred_valid

array([[0.00033939],
       [0.00031637],
       [0.00036191],
       ...,
       [0.00091061],
       [0.00049785],
       [0.00054682]])

# Evaluate process

In [13]:
def evaluate(click, payprice, bidprice, slotprice, budget_litmit=6250 * 1000):
    num_click = 0
    budget = budget_litmit
    cost = 0
    impression = 0
    
    for i in range(len(click)):
        if budget == 0:
            return num_click, cost, impression
        
        if bidprice[i] > payprice[i] and budget > payprice[i] and bidprice[i] > slotprice[i]:
            num_click += click[i]
            budget -= payprice[i]
            impression += 1
            cost += payprice[i]
            
    return num_click, cost, impression
            

# Constant Bid

In [ ]:
bidprice = np.ones_like(payprice) * 78
evaluate(click, payprice, bidprice, slotprice)

# Random Bid

In [ ]:
bidprice = np.random.randint(60,82,size=payprice.shape[0])
evaluate(click, payprice, bidprice, slotprice)

# Linear Bid

In [10]:
bidprice = pred_gl/avgCTR * 67
evaluate(click, payprice, bidprice, slotprice)

(164, 6243736, 119095)

In [11]:
bidprice = pred_esm/avgCTR * 66
evaluate(click, payprice, bidprice, slotprice)

(163, 6127456, 122506)

# Linear Varient 

In [75]:
bidprice = y_pred_valid / avgCTR * 126
threshold = 0.9
mask = pred_esm / avgCTR  < threshold
bidprice[mask] = 0
evaluate(click, payprice, bidprice, slotprice)

(168, 6186218, 99071)

In [86]:
for i in range(100,150):
    bidprice = y_pred_valid / avgCTR * i
    threshold = 0.7
    mask = pred_esm / avgCTR  < threshold
    bidprice[mask] = 0
    print(i)
    print(evaluate(click, payprice, bidprice, slotprice))
    print('------')

100
(165, 5828643, 104569)
------
101
(165, 5876465, 105049)
------
102
(165, 5929419, 105568)
------
103
(165, 5978679, 106047)
------
104
(166, 6030175, 106539)
------
105
(166, 6080261, 107006)
------
106
(166, 6127081, 107463)
------
107
(166, 6176703, 107924)
------
108
(166, 6221630, 108334)
------
109
(164, 6249998, 108405)
------
110
(164, 6249999, 108036)
------
111
(163, 6249999, 107647)
------
112
(160, 6249999, 107281)
------
113


KeyboardInterrupt: 

# ORTB1

In [231]:
c = 145
lamb = 4e-06
bidprice = np.sqrt((c/lamb)*pred_esm + c*c) - c
evaluate(click, payprice, bidprice, slotprice)

(162, 6218439, 130085)

# ORTB2

In [232]:
c = 154
lamb = 6e-06
temp1 = c * lamb
temp2 = np.sqrt(c*c*lamb*lamb+pred_esm*pred_esm) + pred_esm
bidprice = c * (np.power(temp2/temp1, 1.0/3) - np.power(temp1/temp2, 1.0/3))
evaluate(click, payprice, bidprice, slotprice)

(163, 6249997, 130053)

# Gate

In [161]:
threshold = 1.56
bidprice = np.zeros_like(payprice)
mask = (y_pred_valid / avgCTR > threshold).flatten()
bidprice[mask] = 301
evaluate(click, payprice, bidprice, slotprice)

(146, 3459293, 37146)

In [138]:
mask.flatten

array([False, False, False, ..., False, False, False])

# Quardratic

In [125]:
base_bid = 25.2

In [126]:
bidprice = np.exp(y_pred_valid / avgCTR) * base_bid
evaluate(click, payprice, bidprice, slotprice)

(164, 6238622, 143386)

# Combine

In [300]:
c = 105
lamb = 6e-06
bidprice = (np.sqrt((c/lamb)*pred_esm + c*c) - c) * pred_esm / avgCTR
evaluate(click, payprice, bidprice, slotprice)

(165, 6238419, 105085)

In [305]:
c = 42
lamb = 7e-06
temp1 = c * lamb
temp2 = np.sqrt(c*c*lamb*lamb+pred_esm*pred_esm) + pred_esm
bidprice = c * (np.power(temp2/temp1, 1.0/3) - np.power(temp1/temp2, 1.0/3)) * pred_esm / avgCTR
evaluate(click, payprice, bidprice, slotprice)

(163, 5775194, 102581)

## Strategy for test

In [88]:
idx = test['bidid']

In [93]:
bidprice = y_pred_test / avgCTR * 118
threshold = 0.9
mask = y_pred_test / avgCTR  < threshold
bidprice[mask] = 0

In [94]:
result = pd.DataFrame(bidprice, columns=['bidprice'], index = idx)

In [95]:
result

,bidprice
bidid,
366c563de7d90feb9d4dab53e795a93fb3157387,181.254180
29167d4caa719788b5a342dbaa25151d53121f80,560.792323
ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,0.000000
844c2da00d45315f20b748ec131c26ee99a7cbc7,155.085319
c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,209.862921
7493c42f6d8f29d9f414b75c125ce3db40461ef9,0.000000
819fb6958ea48b10430710d009c57d55a31debcb,0.000000
8660774f6c94e32dfbd46add8a9bc94df3c74130,268.538337
ccc4c75ae4fd33526e5ecdae482c45c1dabc94db,0.000000


In [96]:
result.to_csv('../test_bid_price/testing_bidding_price.csv')